In [21]:
import pandas as pd
import os
from thefuzz import fuzz
import re 
import numpy as np

In [5]:
RAW_PATH = "RAW.xlsx"
GEO_PATH = "GEOGRAPHY.xlsx"

In [24]:
df_raw = pd.read_excel(RAW_PATH)
df_raw["DISTRICT"] = df_raw["DISTRICT"].apply(lambda x: "" if x is np.nan else str(x).lower())
df_raw["CITY"] = df_raw["CITY"].apply(lambda x: "" if x is np.nan else re.sub("\d\s", "", str(x)).lower())

In [27]:
df_raw

,DISTRICT,CITY
0,h.hạ hòa,phú thọ
1,tân bình,hồ chí minh
2,thị xã gò công,tỉnh tiền giang
3,dương đông,phú quốc
4,cư kuin,đắk lắk
...,...,...
11072,quận đóng đa,hà nội
11073,quận 1,thành phồ hồ chí minh
11074,"phường ninh khánh, thành phố ninh bình","tỉnh ninh bình, việt nam"
11075,nghia trung,gia nghĩa


In [28]:
df_geo = pd.read_excel(GEO_PATH)
df_geo_low = df_geo.copy()

In [29]:
df_geo_low["CORRECT_DISTRICT"] = df_geo_low["CORRECT_DISTRICT"].apply(lambda x: x.lower())
df_geo_low["CORRECT_CITY"] = df_geo_low["CORRECT_CITY"].apply(lambda x: x.lower())

In [227]:
df_geo_low[df_geo_low["CORRECT_CITY"] == "hà nội" ]

,CORRECT_DISTRICT,CORRECT_CITY
14,ba đình,hà nội
20,ba vì,hà nội
33,bắc từ liêm,hà nội
100,cầu giấy,hà nội
130,chương mỹ,hà nội
161,đan phượng,hà nội
179,đông anh,hà nội
180,đống đa,hà nội
211,gia lâm,hà nội
227,hà đông,hà nội


In [258]:
ind = 193

example = ", ".join([df_raw.loc[ind, "DISTRICT"], df_raw.loc[ind, "CITY"]])
example = re.sub("t*p*[\W\s]*hcm", "thành phố hồ chí minh", example)
example = re.sub("thành phố (?!hồ c)", "", example)
example

'phường láng thượng, quận đống đa, hà nội, việt nam'

2 cases:
1. Missing value
2. Wrong info -> low similarity score 

In [259]:
def compute_fuzzy_scores(example, df_geo_low):
    scores = []
    scores_city = []
    scores_district = []
    scores_reverse = []
    for i, row in df_geo_low.iterrows():
        comp = ", ".join([row["CORRECT_DISTRICT"], row["CORRECT_CITY"]])
        score = fuzz.ratio(comp, example)
        scores.append(score)

        comp_rev = ", ".join([row["CORRECT_CITY"], row["CORRECT_DISTRICT"]])
        scores_reverse.append(fuzz.ratio(comp_rev, example))

        score_c = fuzz.ratio(row["CORRECT_CITY"], example)
        scores_city.append(score_c)

        score_d = fuzz.ratio(row["CORRECT_DISTRICT"], example)
        scores_district.append(score_d)
    return scores, scores_city, scores_district, scores_reverse

In [260]:
scores, scores_city, scores_district, scores_reverse =  compute_fuzzy_scores(example, df_geo_low)

In [261]:
ss = (max(scores), max(scores_city), max(scores_district), max(scores_reverse))

In [262]:
print(max(scores), max(scores_city), max(scores_district), max(scores_reverse))
print(df_geo.iloc[scores.index(max(scores)), :])
print("CITY:", df_geo.loc[scores_city.index(max(scores_city)), "CORRECT_CITY"])
print("DISTRICT:", df_geo.loc[scores_district.index(max(scores_district)), "CORRECT_DISTRICT"])
print(df_geo.loc[scores_reverse.index(max(scores_reverse)), :])

47 31 37 45
CORRECT_DISTRICT    Thường Tín
CORRECT_CITY            Hà Nội
Name: 602, dtype: object
CITY: Thành phố Hồ Chí Minh
DISTRICT: Phan Rang – Tháp Chàm
CORRECT_DISTRICT    Hàm Thuận Nam
CORRECT_CITY           Bình Thuận
Name: 245, dtype: object


### Run whole file

In [232]:
res_district = []
res_city = []
notes = []

In [233]:
for ind in range(len(df_raw)):
    example = ", ".join([df_raw.loc[ind, "DISTRICT"], df_raw.loc[ind, "CITY"]])
    example = re.sub("t*p*[\W\s]*hcm", "thành phố hồ chí minh", example)
    example = re.sub("thành phố (?!hồ c)", "", example)
    scores, scores_city, scores_district, scores_reverse =  compute_fuzzy_scores(example, df_geo_low)
    ss = (max(scores), max(scores_city), max(scores_district), max(scores_reverse))
    if max(ss) <= 50:
        res_district.append("")
        res_city.append("")
        notes.append("## UNRECOGNISED ##")
        continue
    res_row = df_geo.loc[scores.index(max(scores)), :]
    res_row_rev = df_geo.loc[scores_reverse.index(max(scores_reverse)), :]
    if max(ss) == ss[0]:
        res_district.append(res_row["CORRECT_DISTRICT"])
        res_city.append(res_row["CORRECT_CITY"])
    elif max(ss) == ss[1] or max(ss) == ss[2]:
        city = df_geo.loc[scores_city.index(max(scores_city)), "CORRECT_CITY"]
        district = df_geo.loc[scores_district.index(max(scores_district)), "CORRECT_CITY"]
        if ss[1] == ss[2]:
            res_city.append(city)
            res_district.append(district)
        elif ss[1] > ss[2]:
            res_district.append("")
            res_city.append(city)
        else:
            res_district.append(district)
            res_city.append("")
    else:
        res_district.append(res_row_rev["CORRECT_DISTRICT"])
        res_city.append(res_row_rev["CORRECT_CITY"])
    if max(ss) <= 65:
        notes.append("## RECHECK ##")
    else:
        notes.append("")

In [234]:
df = pd.DataFrame.from_dict({"CORRECTED_DISTRICT": res_district, "CORRECTED_CITY": res_city, "NOTES": notes})

In [235]:
df

,CORRECTED_DISTRICT,CORRECTED_CITY,NOTES
0,Hạ Hòa,Phú Thọ,
1,Tân Bình,Thành phố Hồ Chí Minh,
2,Gò Công,Tiền Giang,
3,Phú Quốc,Kiên Giang,
4,Cư Kuin,Đắk Lắk,
...,...,...,...
11072,Đống Đa,Hà Nội,
11073,Quận 1,Thành phố Hồ Chí Minh,
11074,Ninh Bình,Ninh Bình,## RECHECK ##
11075,Gia Nghĩa,Đắk Nông,


In [236]:
df.to_excel("RAW_CORRECTED.xlsx", sheet_name="Corrected")